In [ ]:
pip install autocorrect

In [ ]:
pip install pyspellchecker

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
import numpy as np
import pandas as pd
from spacy import displacy
from autocorrect import Speller
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
nlp = spacy.load("en_core_web_sm")
# nlp = spacy.load('en_core_web_lg')

In [ ]:
short_forms1 = {
    "SOB": "Shortness of Breath",
    "CP": "Chest Pain",
    "HA": "Headache",
    "N/V": "Nausea/Vomiting",
    "F/C": "Fever/Chills",
    "D/C": "Discharge (from a wound)",
    "URI": "Upper Respiratory Infection",
    "UTI": "Urinary Tract Infection",
    "BM": "Bowel Movement",
    "DVT": "Deep Vein Thrombosis",
    "HTN": "Hypertension",
    "DM": "Diabetes Mellitus",
    "COPD": "Chronic Obstructive Pulmonary Disease",
    "RA": "Rheumatoid Arthritis",
    "GERD": "Gastroesophageal Reflux Disease",
    "OTC": "Over-the-Counter",
    "Rx": "Prescription",
    "NSAID": "Nonsteroidal Anti-Inflammatory Drug",
    "MRI": "Magnetic Resonance Imaging",
    "CT": "Computed Tomography",
    "CBC": "Complete Blood Count",
    "ECG/EKG": "Electrocardiogram",
    "IV": "Intravenous",
    "PCP": "Primary Care Physician",
    "ER": "Emergency Room",
    "ICU": "Intensive Care Unit",
    "AMA": "Against Medical Advice",
    "NPO": "Nothing by Mouth",
    "meds": "medications",
    "doc": "doctor",
    "xray": "X-ray",
    "mri": "magnetic resonance imaging",
    "er": "emergency room",
    "vet": "veterinarian",
    "gp": "general practitioner",
    "hmd": "Help, Medical Emergency!",
    "sos" : "Save Our Souls",
    "tia": "This is an emergency",
    "code blue": "Medical emergency requiring resuscitation" ,
    "rta": "Requesting Treatment ASAP",
    "ama": "Against Medical Advice",
    "icu": "Intensive Care Unit",
    "dnr": "Do Not Resuscitate",
    "od": "Overdose",
    "ptsd": "Post-Traumatic Stress Disorder",
    "ems": "Emergency Medical Services",
    "apap": "As Per Approved Protocol",
    "iv": "Intravenous",
    "ehr": "Electronic Health Record",
    "rx": "Prescription",
    "npo": "Nothing by Mouth",
    "l&d": "Labor and Delivery",
    "pmh": "Past Medical History",
    "asap": "As Soon As Possible",
    "loc": "Loss of Consciousness",
    "pe": "Pulmonary Embolism",
    "cdc": "Centers for Disease Control and Prevention",
    "bpm": "Beats Per Minute",
    "emt": "Emergency Medical Technician",
    "afib": "Atrial Fibrillation",
    "abo": "Already Been Operated",
    "tbi": "Traumatic Brain Injury",
    "gerd": "Gastroesophageal Reflux Disease",
    "ent": "Ear, Nose, and Throat",
    "ra": "Rheumatoid Arthritis"
}

In [ ]:
#forming final short forms dictionary
short_forms = {}
for i in short_forms1:
  short_forms[i.lower()]=short_forms1[i].lower()
# short_forms

In [ ]:
#Storing the short forms in a binary file (pickle file)
import pickle
f=open("short_forms.pkl",'wb')
pickle.dump(short_forms,f)
f.flush()
f.close()

In [ ]:
def socialmediacheck(word):
  if word in short_forms:
    return short_forms[word]
  else:
    return None

In [ ]:
def spellchecker(word):
  word=word.lower()
  # spell = Speller() #autocorrect lib func
  spell = SpellChecker() #pyspellchecker lib func
  t1=socialmediacheck(word)
  if t1!=None:
    return t1
  else:
    return spell.correction(word)

In [ ]:
def convert_time(time_str):

    # Split the time string by ':' and space to separate hours and minutes
    parts = time_str.split(':')
    hours, minutes = int(parts[0]), int(parts[1].split()[0])

    # Check if it's AM or PM and adjust the hours if necessary
    if "pm" in time_str.lower() and hours != 12:
        hours += 12
    elif "am" in time_str.lower() and hours == 12:
        hours = 0

    # Convert hours and minutes to a 4-digit string
    time_24hr = f"{hours:02}{minutes:02}"

    return time_24hr

In [ ]:
# text=input("Enter input text:")
# text="Hello everyonee, myshelf Gaurang Dixit, I'm having nausea. Book me an appointment ASAP on 29th of October at 6pm or else you willl go to jail and Gaurang Dixit will be sad. "
text="Hello my name is Gaurang Dixit and I keep sneezing, and my eyes don't quit dripping. "
doc=nlp(text)

In [ ]:
#getting all named entities (built in entites in SpaCy)
ent_set=set({})
for ent in doc.ents:
  if ent.label_ == 'PERSON':
    ent_set.add(ent.text)

ent_set

{'Gaurang Dixit'}

In [ ]:
#testing for short forms
for i in doc:
  print(i.text,socialmediacheck(i.text.lower()),sep=":")

Hello:None
my:None
name:None
is:None
Gaurang:None
Dixit:None
and:None
I:None
keep:None
sneezing:None
,:None
and:None
my:None
eyes:None
do:None
n't:None
quit:None
dripping:None
.:None


In [ ]:
#replacing all entities with a single word (for easy searching and storing those abbr in a dict for later so as to get back the original string)
ent_all={}
ct=-1 #key value for entities
for i in ent_set:
  ent_all[str(ct)]=i
  text=text.replace(str(i),str(ct))
  ct-=1

# text
#recreating doc object
doc=nlp(text)
ent_all

{'-1': 'Gaurang Dixit'}

In [ ]:
#creating final corrected and short form free text sentence
text_final=' '
for tok in doc:
  if tok.text not in ent_all:
    text_final+=spellchecker(tok.text)
  else:
    text_final+=tok.text
  text_final+=" "

for i in ent_all:
  text_final=text_final.replace(i,ent_all[i])
print("Final text is:",text_final)

#---------------------------------------------------------------
#auto correct and social media pre-processing done

Final text is:  hello my name is Gaurang Dixit and i keep sneezing , and my eyes do not quit dripping . 


In [ ]:
#all the preprocessing of the text
def preprocess(text):
  doc=nlp(text)

  #getting all named entities (built in entites in SpaCy)
  ent_set=set({})
  for ent in doc.ents:
    if ent.label_ in ['PERSON','TIME']:
      ent_set.add(ent.text)
  # print("Ent set:")
  # for i in ent_set:
  #   print(i)

  #replacing all entities with a single word (for easy searching and storing those abbr in a dict for later so as to get back the original string)
  ent_all={}
  ct=-1 #key value for entities
  for i in ent_set:
    ent_all[str(ct)]=i
    text=text.replace(str(i),str(ct))
    ct-=1
  # print("Ent all:")
  # for i in ent_all:
  #   print(i,":",ent_all[i])
  # print("Text:",text)
  #creating final corrected and short form free text sentence

  doc=nlp(text)
  text_final=' '
  for tok in doc:
    if tok.text not in ent_all:
      text_final+=spellchecker(tok.text)
    else:
      text_final+=tok.text
    text_final+=" "

  for i in ent_all:
    text_final=text_final.replace(i,ent_all[i])
  # print("Final text is:",text_final)
  return text_final
#---------------------------------------------------------------

In [ ]:
#Now classification task begins

In [ ]:
#Getting text and label from dataset
df=pd.read_csv('Symptom2Disease.csv')
labels = df['label'].tolist()
text = df['text'].tolist()

In [ ]:
labels

In [ ]:
#pre-processing dataset text
text_final = []
for i in text:
  doc1 = nlp(i)
  sent= [tok.text.lower() for tok in doc1 if not tok.is_punct and not tok.is_space]
  sent_final= ' '.join(token for token in sent if token not in nlp.Defaults.stop_words)
  text_final.append(sent_final)

In [ ]:
#label substituting (pre-preprocessing dataset labels)
label_to_no={}
no_to_label={}
ct=0
for i in range(len(labels)):
  labels[i]=labels[i].lower()
  if labels[i] not in label_to_no:
    label_to_no[labels[i]]=ct
    no_to_label[ct]=labels[i]
    labels[i]=ct
    ct+=1
  else:
    labels[i]=label_to_no[labels[i]]

labels=np.array([i for i in labels])
no_to_label
# label_to_no

In [ ]:
#experimenting------------------------

#here using vstack is the same as creating an array by using append function, but we use it to maintain standards and remove any confusion
#becauase there is another function hstack which joins array horizontally, i.e. adding row1 of mat1 to row1 of mat2, then row2 of mat1 to row2 of mat2,....
#whereas in vstack it is simple append like first all rows of mat1, then all rows of mat2 one by one as happens in append (see diagram in net to understand better)
feature_matrix = np.vstack([token.vector for token in doc])
# feature_matrix= [list(i.vector[:3]) for i in doc]
feature_matrix

#experimenting------------

In [ ]:
#creating feature matrix for training data (using word vector of spacy)
feature_matrix = []

# Loop through the text data
for i in text_final:
    # Process the text with spaCy
    doc1 = nlp(i)
    feature_matrix.append(doc1.vector)

# Convert the list of feature matrices to a single numpy array
feature_matrix = np.array(feature_matrix)

In [ ]:
#creating feature matrix for training data (using tf-idf of sk-learn)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
feature_matrix = vectorizer.fit_transform(text_final)
# feature_matrix = tfidf_matrix.toarray()
# tfidf_array

In [ ]:
#dividing data into train and test data
text_train, text_test, labels_train, labels_test = train_test_split(feature_matrix, labels, test_size=0.2, random_state=41)

In [ ]:
#experimenting-------------
x=list(labels_train)
for i in range(0,24):
  print(i,":",x.count(i))
#--------------------------------

0 : 40
1 : 42
2 : 43
3 : 41
4 : 41
5 : 45
6 : 38
7 : 41
8 : 43
9 : 43
10 : 35
11 : 43
12 : 40
13 : 45
14 : 38
15 : 33
16 : 39
17 : 44
18 : 36
19 : 40
20 : 37
21 : 41
22 : 38
23 : 34


In [ ]:
#training model
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
# model=LogisticRegression()
# model = MultinomialNB()
# model = SVC(kernel='linear')
# model = RandomForestClassifier(n_estimators=100, random_state=42)
model = MLPClassifier(hidden_layer_sizes=(100, ), max_iter=1000, random_state=42, activation='tanh')

model.fit(text_train,labels_train)

In [ ]:
from sklearn.metrics import classification_report
label_predict=model.predict(text_test)
report_dict=classification_report(labels_test,label_predict)
print(report_dict)

In [ ]:
# nlp_l=spacy.load('en_core_web_lg')
user_text="Hello my name is Gaurang Dixit and I have loose motion and pain in the stomach. Book me an appointment at 11:40 pm"
# user_text="Hi myself Vishwak, I am having cough,cold and high fever."
user_text_final=preprocess(user_text)
print("Final text is:",user_text_final)
doc=nlp(user_text_final)
tokens= [tok.text.lower() for tok in doc if not tok.is_punct and not tok.is_space]
tokens= [token for token in tokens if token not in nlp.Defaults.stop_words]
doc=nlp(' '.join(i for i in tokens)) #final nlp object
# tokens
#for now not doing lemmatization

Final text is:  hello my name is Gaurang Dixit and i have loose motion and pain in the stomach . book me an appointment at 11:40 pm 


In [ ]:
print(doc.text)

hello gaurang dixit loose motion pain stomach book appointment 11:40 pm


In [ ]:
#testing on the given text
# guess=model.predict(np.array([doc.vector,doc.vector]))
guess=model.predict(vectorizer.transform([doc.text]))
no_to_label[guess[0]]

'typhoid'

In [ ]:
doc=nlp(user_text_final)
for i in doc.ents:
  if i.label_=='TIME':
    print(i.text,convert_time(i.text))

11:40 pm 2340


In [ ]:
label_to_no

{'psoriasis': 0,
 'varicose veins': 1,
 'typhoid': 2,
 'chicken pox': 3,
 'impetigo': 4,
 'dengue': 5,
 'fungal infection': 6,
 'common cold': 7,
 'pneumonia': 8,
 'dimorphic hemorrhoids': 9,
 'arthritis': 10,
 'acne': 11,
 'bronchial asthma': 12,
 'hypertension': 13,
 'migraine': 14,
 'cervical spondylosis': 15,
 'jaundice': 16,
 'malaria': 17,
 'urinary tract infection': 18,
 'allergy': 19,
 'gastroesophageal reflux disease': 20,
 'drug reaction': 21,
 'peptic ulcer disease': 22,
 'diabetes': 23}

In [ ]:
#Dictionary 1 disease to specialisation
disease_specialization = {
    'psoriasis': 'Dermatology',
    'impetigo': 'Dermatology',
    'pneumonia': 'Pulmonology',
    'dengue': 'Virology',
    'common cold': 'Virology',
    'varicose veins':'Dermatology',
    'typhoid':'Gastroenterology',
    'chicken pox':'Dermatology',
    'fungal infection':'General',
    'dimorphic hemorrhoids':'Gastroenterology',
    'arthritis':'Orthology',
    'acne':'Dermatology',
    'bronchial asthma':'Pulmonology',
    'hypertension':'General',
    'migraine':'General',
    'cervical spondylosis':'Orthology',
    'jaundice':'Gastroenterology',
    'malaria':'Virology',
    'urinary tract infection':'General',
    'allergy':'General',
    'gastroesophageal reflux disease':'Gastroenterology',
    'drug reaction':'General',
    'peptic ulcer disease':'Gastroenterology',
    'diabetes':'General',
}


specialization_doctors = {
    'Dermatology': {
        '0': 'Dr.Ramesh',
        '1': 'Dr.Michael',
    },
    'Pulmonology': {
        '2': 'Dr.Chitambaram',
        '3': 'Dr.Richard',
    },
    'Virology': {
        '4': 'Dr.Prateek Kumar',
        '5': 'Dr.Rory',
    },
    'Gastroenterology': {
        '6': 'Dr.Mishra',
        '7': 'Dr.Stephen',
    },
    'Orthology': {
        '8': 'Dr.Sandeep',
        '9': 'Dr.Robert',
    },
    'General': {
        '10': 'Dr.Vinod',
        '11': 'Dr.Jeffery',
    },
}

doctor_availability = {
    '0': ['1130'],
    '1': ['1000', '1530'],
    '2': [ '1200'],
    '3': [ '1130'],
    '4': ['0800', '1030', '1600'],
    '5': ['0945', '1315'],
    '6': ['1000', '1115'],
    '7': ['0915', '1115', '1315'],
    '8': ['1000', '1115'],
    '9': ['1030', '1800'],
    '10': ['0845', '1100'],
    '11': ['1145', '1545'],
}


In [ ]:
import pickle
f=open("Disease_DB.pkl",'wb')
pickle.dump(disease_specialization,f)
pickle.dump(specialization_doctors,f)
pickle.dump(doctor_availability,f)
f.flush()
f.close()

In [ ]:
f=open("Disease_DB.pkl",'rb+')

In [ ]:
#experimenting
# lst=[]
# f.seek(0,0)
# try:
#   while True:
#     a=pickle.load(f)
#     lst.append(a)
# except:
#   print("hello")

# for i in lst:
#   print(type(i))